## Overview
This python file generates outputs for the following toolkits:-
1. ShareDay Topline
2. Executive Toolkit
3. Executive Summary
4. Category Toolkit Oral Care
5. Category Toolkit Personal Care
6. Category Toolkit Home Care

**Importing all the libraries**

In [ ]:
import pandas as pd;
import numpy as np;
import os;
import time;
import datetime;
import warnings;
warnings.filterwarnings('ignore');

**Accessing Input Files**

In [ ]:
# Assigning path of the folder where all "Line Monthly" data files are stored
path = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/09. Personal Work/Madhwan/Line Monthly"
files = os.listdir(path)

# Selecting only excel files from all the files available in the above path
files_xls = [f for f in files if f[-4:] == 'xlsx'] 
print("Total number of files in the folder are ",len(files_xls))

**Appending the read files in rows and storing them in a single dataframe** named ***Line_Monthly***.

Printing number of columns and number of facts in each file.
Also checking the execution time of the code

In [ ]:
time_start= time.time() # giving the current time

Line_Monthly= pd.DataFrame() # creating a dataframe name Line_Monthly

j=0
for f in files_xls:
    
    data = pd.read_excel(path+"/"+f) # reading the file
    
    # priniting number of facts and number of columns of the read file
    print("\033[1m" + files_xls[j] + "\033[0m",":->\n# of Facts are ",len(set(data["Periods"])),"\t\t # of columns are ",len(data.columns))  
    
    # appending the read files in "Line_Monthly" dataframe
    Line_Monthly = pd.concat([Line_Monthly,data],axis=0,ignore_index=True)
    j=j+1
    
time_end= time.time();# giving the current time

Backup= Line_Monthly.copy() # storing Line_Monthly dataframe into anoter dataframe for backup purpose

# printing the time it takes to run this block of code 
print("execution time is ", time_end-time_start)

**Removing Duplicate Rows**

In [ ]:
# Dropping the Duplicate rows
lenth_df=len(Line_Monthly)
Line_Monthly.drop_duplicates(subset=["Markets","Products","Periods"],keep='last',inplace=True)
print("Number of duplicated rows are",lenth_df-len(Line_Monthly),"\nShape of Line_Monthly is ",Line_Monthly.shape)

**Replacing Facts names** (Here column "Periods" denotes the fact column)

In [ ]:
set(Line_Monthly['Periods'])

In [ ]:
# Replacing "- $ -" with "-" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("$","temp")
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("- temp -","-")
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("temp","$")

# Replacing "CP FORM" with "CP CATEGORY" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("CP FORM","CP CATEGORY")

# Replacing "CP USE" with "CP CATEGORY" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("CP USE","CP CATEGORY")

# Replacing "CP LDL AND FOAM" with "CP CATEGORY" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("CP LDL AND FOAM","CP CATEGORY")

# Replacing "CP LDL AND FM X DISHAIDS/WPS/SPR" with "CP CATEGORY" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("CP LDL AND FM X DISHAIDS/WPS/SPR","CP CATEGORY")

# Replacing "CP MEGA CATEGORY" with "CP CATEGORY" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("CP MEGA CATEGORY","CP CATEGORY")

# Replacing "CP BATTERY AND HYBRID" with "CP CATEGORY" in column "Periods"
Line_Monthly["Periods"]=Line_Monthly["Periods"].str.replace("CP BATTERY AND HYBRID","CP CATEGORY")

# Market name replaced
# Replacing "Walgreens Corp Total TA" with "Walgreens Total TA" in column "Markets"
Line_Monthly["Markets"]=Line_Monthly["Markets"].str.replace("Walgreens Corp Total TA","Walgreens Total TA")

In [ ]:
set(Line_Monthly["Periods"]);

**Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

In [ ]:
new_col_names=[]
for i in Line_Monthly.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))
        

**Checking the duplicacte column in** ***Line_Monthly***

In [ ]:
if(len(new_col_names)!=len(set(new_col_names))):
    print("\033[1;31m"+"*******************************ERROR*******************************\n\t\t\tWe have duplicate columns\n\t\t\tPlease recheck raw data files\n*******************************ERROR*******************************")

Renaming the column names of ***Line_Monthly***

In [ ]:
# Renaming the column names
Line_Monthly.columns=new_col_names

Line_Monthly.columns

**Adding Missing Quarter**

In [ ]:
# Getting 20 days before month and year value

now=datetime.datetime.now()  # current date & time
delay= datetime.timedelta(days=20) # 20 days delay

a= now - delay # 20 days ago time period
month_number=a.month # 20 days ago month number 
year=str(a.year-2)[-2:] # 2 year & 20 days ago year number

print("Month ->",month_number)
print("Year ->",year)

# Adding Quarter's columns
if( month_number >= 1 & month_number <= 3):
    Line_Monthly['AMJ '+year]=""
    Line_Monthly['JAS '+year]=""
    Line_Monthly['OND '+year]=""
elif ( month_number >= 4 & month_number <= 6 ):
    Line_Monthly['JAS '+year]=""
    Line_Monthly['OND '+year]=""
elif ( month_number >= 7 & month_number <= 9  ):
    Line_Monthly['OND '+year]=""

**Time period mapping**

**Manual Step**

This mapping needs to be changed every year when new year data arrives

In [ ]:
time_period_map= pd.DataFrame({'Connect Time Period':['Markets', 'Products', 'Periods','Cal Yr 2020','Cal Yr 2019','Cal Yr 2018','Cal Yr 2017','YTD','YTD YA','Latest 13 Wks','Latest 26 Wks','OND 21','JAS 21','AMJ 21','JFM 21','OND 20','JAS 20','AMJ 20','JFM 20','OND 19','JAS 19','AMJ 19','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20'],
                               'Final Name':['Markets', 'Products', 'Periods','FY 2020','FY 2019','FY 2018','FY 2017','YTD 2021','YTD 2020','Latest 13 Weeks','Latest 26 Weeks','Q4 2021','Q3 2021','Q2 2021','Q1 2021','Q4 2020','Q3 2020','Q2 2020','Q1 2020','Q4 2019','Q3 2019','Q2 2019','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20']
                              ,'Order':range(1,47)})

time_period_map.head(50)

Changing **column names** of ***Line_Monthly*** and **reordering** them based on the time_period_mapping

In [ ]:
# assigning variables

LM_column= list(Line_Monthly.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column=[]
for i in range(0,len(LM_column)):
    for j in range(0,len(time_connect)):
        if LM_column[i]==time_connect[j]:
            new_column.append(time_final[j])
            
# assigning new column names to Line_Monthly
Line_Monthly.columns = new_column

# reordering column names
my_list =time_period_map.iloc[:, 1]
my_list1=Line_Monthly.columns
result = [x for x in my_list if x in my_list1]
Line_Monthly=Line_Monthly[result]

print(Line_Monthly.columns)

In [ ]:
Line_Monthly.head(3)

creating ***movecol*** function to **rearranging** the columns of a dataframe

In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place=''):
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

**Splitting "Products" column**

Creating 5 new columns **"ShareBasis","Manufacturer","Franchise","Brand","SubBrand"** from **"Products"** column by splitting 

In [ ]:
# assigning new_column names 
New_Columns=['ShareBasis','Manufacturer','Franchise','Brand','SubBrand']

# Splitiing column "Product" in 5 new columns "New_Columns"
Line_Monthly[New_Columns] = Line_Monthly['Products'].str.split('|',n=4,expand=True)

# Filling NAN with blank values
Line_Monthly[New_Columns]=Line_Monthly[New_Columns].fillna(value="")

# Moving columns after "Market"
Line_Monthly = movecol(Line_Monthly, 
             cols_to_move=New_Columns, 
             ref_col='Markets',
             place='After')

# showing dataframe Line_Monthly
Line_Monthly.head(3)

In [ ]:
# Changing Column Name "Periods" to "Facts"
Line_Monthly.rename(columns={"Periods":"Facts"},inplace=True)

# Adding a "KEY" column
Line_Monthly["Key"]=Line_Monthly["Markets"]+Line_Monthly["Products"]+Line_Monthly["Facts"]

# moving "KEY" column to start of the column
Line_Monthly = movecol(Line_Monthly, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

# showing dataframe Line_Monthly
Line_Monthly.head(3)

**Strip Columns and Change Datatype to numeric**

Strip starting 9 columns 

Changing datatype to numeric after 9 columns

In [ ]:
# Choose starting 9 columns
cols=list(Line_Monthly.columns[0:9])

# Doing strip to starting 9 columns
Line_Monthly[cols]=Line_Monthly[cols].apply(lambda x : x.str.strip())

# Changing all columns after "Facts" into numeric datatype
Line_Monthly.iloc[:,9:]= Line_Monthly.iloc[:,9:].apply(pd.to_numeric)

#Backup
Line_Monthly__strip_copy=Line_Monthly.copy()

**Category column**

Creating a extra column name "Category" from column "ShareBasis"

In [ ]:
df1= pd.DataFrame(Line_Monthly["ShareBasis"])

In [ ]:
def fun_category(x):
    if x == 'TOOTHPASTE' or x == 'WHITENING SYS':
        return "TP"
    elif x=='POUR APC+WW' or x=="WOOD" or x== 'ABRASIVE' or x== "APC+WW" or x=='FLOOR WAX' or x=='OUTDOOR' or x=='DRY WIPES' or x=='SPRAY':
        return "HHC"
    elif x=='MANUAL':
        return 'MTB'
    elif x=='POWER' or x=='PORTABLE TB' or x == 'ELECTRIC' or x == 'BATTERY+HYBRID' or x == 'BATTERY':
        return "PTB"
    elif x=='BODYWASH':
        return "BW"
    elif x=='MOUTHWASH':
        return 'MW'
    elif x=='BAR':
        return "BAR"
    elif x=='LIQUID':
        return 'LHS'
    elif x=='FABRIC SOFTENERS' or x == 'BEAD FS' or x== 'LIQUID FS' or x == "SHEETS" or x=="SCENT BOOSTERS" or x == 'MFR Sun Products' :
        return "FS"
    elif x=='LDL+FOAM' or x == 'MFR Sun Products LDL' or x == "LDL/DISHAIDS/WPS/FM/SPRAYS":
        return 'LDL'
    else:
        return 'N/A'
    # Add elif statements....

df1['Category']= df1['ShareBasis'].apply(fun_category)

Line_Monthly['Category']=df1['Category']

#Moving column "Category" after column "ShareBasis"

Line_Monthly = movecol(Line_Monthly, 
             cols_to_move=['Category'], 
             ref_col='ShareBasis',
             place='After')

# showing dataframe Line_Monthly
Line_Monthly.head(3)

In [ ]:
Line_Monthly.info()

## UAP DATA

**Reading input file**

In [ ]:
UAP_Data=pd.read_excel("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/UAP_ADD_Data/UAP AOD Report.xlsx")

In [ ]:
# del UAP_Data['Jan 21']

**Replacing Facts names** (Here column "Periods" denotes the fact column)

In [ ]:
# Replacing "- Category" with "- CP CATEGORY" in column "Periods"
UAP_Data["Periods"]=UAP_Data["Periods"].str.replace("- Category","- CP CATEGORY")

# Replacing "- Sub Category" with "- CP CATEGORY" in column "Periods"
UAP_Data["Periods"]=UAP_Data["Periods"].str.replace("- Sub Category","- CP CATEGORY")

# Replacing "- Super Category" with "- CP CATEGORY" in column "Periods"
UAP_Data["Periods"]=UAP_Data["Periods"].str.replace("- Super Category","- CP CATEGORY")

**Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

In [ ]:
new_col_names=[]
for i in UAP_Data.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))

        
## assigning new column names
UAP_Data.columns = new_col_names

# assigning variables
LM_column= list(UAP_Data.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column=[]
for i in range(0,len(LM_column)):
    for j in range(0,len(time_connect)):
        if LM_column[i]==time_connect[j]:
            new_column.append(time_final[j])

# assigning new column names
UAP_Data.columns = new_column

In [ ]:
# Changing Column Name "Periods" to "Facts"
UAP_Data.rename(columns={"Periods":"Facts"},inplace=True)

# showing dataframe UAP_Data
UAP_Data.head(3)

**Shifting Timeperiod to rows and Facts to columns**

In [ ]:
# storing UAP_Data in df
df=UAP_Data.copy()

# selecting time periods
Time_Period=list(df.columns[3:])

# applying melt (both Time_Period and Facts will come on rows)
df1=pd.melt(df, id_vars =['Markets', 'Products', 'Facts'], value_vars =Time_Period,var_name="Time_Period")

# changing datatype of "value" into "numeric"
df1["value"]=df1["value"].apply(pd.to_numeric)

# applying pivot_table ( Time_Period will come on rows and Facts will come on columns)
df2=df1.pivot_table(index=['Markets', 'Products','Time_Period'],columns='Facts')
df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

# showing dataframe UAP_Data
df2.head(3)

Creating **two calculated** columns **"EQ Shr - CP CATEGORY"** and **"EQ Shr Chg YA - CP CATEGORY"**

In [ ]:
# creating a new column by concatinating columns "Markets" and "Time_Period"
df2["Market_Period"]=df2["Markets"]+df2["Time_Period"]

# creating a blank dataframe name temp_uap
temp_uap=pd.DataFrame()


for i in list(set(df2["Market_Period"])):
    
    # filtering the dataframe for each Market_Period combination
    df3=df2[df2["Market_Period"]==i]
    
    # getting TTL UAP's EQ value for Market_Period combination
    var=int(df3[df3["Products"]=="TTL UAP"]["EQ"])
    
    # Creating calculated columns
    df3["EQ Shr - CP CATEGORY"]=(df3["EQ"]/var)*100
    df3["EQ LY"]=(df3["EQ"]-df3["EQ Chg YA"])

    # getting TTL UAP's EQ LY value for Market_Period combination
    var_LY=int(df3[df3["Products"]=="TTL UAP"]["EQ LY"])
    
    # Creating calculated columns
    df3["EQ Shr LY"]=(df3["EQ LY"]/var_LY)*100
    df3["EQ Shr Chg YA - CP CATEGORY"]=(df3["EQ Shr - CP CATEGORY"]-df3["EQ Shr LY"])
    
    # concat dataframe df3 for each Market_Period Combination
    temp_uap=pd.concat([temp_uap,df3],axis=0)
    
#loop Ends#

# Deleting the unwanted columns
del temp_uap['Market_Period']
del temp_uap['EQ LY']
del temp_uap['EQ Shr LY']
   
# showing dataframe temp_uap
temp_uap.head(3)

In [ ]:
# selecting facts
Facts=list(temp_uap.columns[3:])

# applying melt (both Time_Period and Facts will come on rows)
df1=pd.melt(temp_uap, id_vars =['Markets', 'Products', 'Time_Period'], value_vars =Facts,var_name="Facts")

# changing datatype of "value" into "numeric"
df1["value"]=df1["value"].apply(pd.to_numeric)

# applying pivot_table ( Time_Period will come on columns and Facts will come on rows )
df2=df1.pivot_table(index=['Markets', 'Products','Facts'],columns='Time_Period')
df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

# showing dataframe df2
df2.head()

In [ ]:
# saving dataframe df2 to UAP_Data_final
UAP_Data_final=df2.copy()

# Creating "ShareBasis" column
UAP_Data_final["ShareBasis"]="UAP"

# Creating "Category" column
UAP_Data_final["Category"]="UAP"

# Creating "Manufacturer" column by replacing "MFR " to blank from "Products column"
UAP_Data_final["Manufacturer"]=[x.replace("MFR ","") if "MFR" in x else "" for x in UAP_Data_final["Products"]]

# Creating "Key" column by conacatinating columns "Markets","Products","Facts"
UAP_Data_final["Key"]=UAP_Data_final["Markets"]+UAP_Data_final["Products"]+UAP_Data_final["Facts"]


## ADD DATA

**Reading input file**

In [ ]:
ADD_Data=pd.read_excel("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/UAP_ADD_Data/ADD AOD Report.xlsx")

In [ ]:
ADD_Data.columns

In [ ]:
del ADD_Data['Jan 21']

**Replacing Facts names** (Here column "Periods" denotes the fact column)

In [ ]:
# Replacing "- Category" with "- CP CATEGORY" in column "Periods"
ADD_Data["Periods"]=ADD_Data["Periods"].str.replace("- Category","- CP CATEGORY")

# Replacing "- Sub Category" with "- CP CATEGORY" in column "Periods"
ADD_Data["Periods"]=ADD_Data["Periods"].str.replace("- Sub Category","- CP CATEGORY")

# Replacing "- Super Category" with "- CP CATEGORY" in column "Periods"
ADD_Data["Periods"]=ADD_Data["Periods"].str.replace("- Super Category","- CP CATEGORY")

**Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

In [ ]:
new_col_names=[]
for i in ADD_Data.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))

        
## assigning new column names
ADD_Data.columns = new_col_names

# assigning variables
LM_column= list(ADD_Data.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column=[]
for i in range(0,len(LM_column)):
    for j in range(0,len(time_connect)):
        if LM_column[i]==time_connect[j]:
            new_column.append(time_final[j])

#assigning new column names
ADD_Data.columns = new_column

In [ ]:
# Changing Column Name "Periods" to "Facts"
ADD_Data.rename(columns={"Periods":"Facts"},inplace=True)

# showing dataframe UAP_Data
ADD_Data.head(3)

In [ ]:
# storing ADD_Data in df
df=ADD_Data.copy()

# selecting time periods
Time_Period=list(df.columns[3:])

# applying melt (both Time_Period and Facts will come on rows)
df1=pd.melt(df, id_vars =['Markets', 'Products', 'Facts'], value_vars =Time_Period,var_name="Time_Period")

# changing datatype of "value" into "numeric"
df1["value"]=df1["value"].apply(pd.to_numeric)

# applying pivot_table ( Time_Period will come on rows and Facts will come on columns)
df2=df1.pivot_table(index=['Markets', 'Products','Time_Period'],columns='Facts')
df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

# showing dataframe df2
df2.head()

Creating **two calculated** columns **"EQ Shr - CP CATEGORY"** and **"EQ Shr Chg YA - CP CATEGORY"**

In [ ]:
# creating a new column by concatinating columns "Markets" and "Time_Period"
df2["Market_Period"]=df2["Markets"]+df2["Time_Period"]

# creating a blank dataframe name temp_uap
temp_add=pd.DataFrame()

for i in list(set(df2["Market_Period"])):
    
    # filtering the dataframe for each Market_Period combination
    df3=df2[df2["Market_Period"]==i]
    
    # getting TTL AUTODISH's EQ value for Market_Period combination
    var=int(df3[df3["Products"]=="TTL AUTODISH"]["EQ"])
    
    # Creating calculated columns
    df3["EQ Shr - CP CATEGORY"]=(df3["EQ"]/var)*100
    df3["EQ LY"]=(df3["EQ"]-df3["EQ Chg YA"])
    
    # getting TTL UAP's EQ LY value for Market_Period combination
    var_LY=int(df3[df3["Products"]=="TTL AUTODISH"]["EQ LY"])
    
    # Creating calculated columns
    df3["EQ Shr LY"]=(df3["EQ LY"]/var_LY)*100
    df3["EQ Shr Chg YA - CP CATEGORY"]=(df3["EQ Shr - CP CATEGORY"]-df3["EQ Shr LY"])
    
    # concat dataframe df3 for each Market_Period Combination
    temp_add=pd.concat([temp_add,df3],axis=0)
    
#loop Ends#

del temp_add['Market_Period']
del temp_add['EQ LY']
del temp_add['EQ Shr LY']
    
# showing dataframe temp_uap
temp_add.head(3)    

In [ ]:
# selecting facts
Facts=list(temp_add.columns[3:])

# applying melt (both Time_Period and Facts will come on rows)
df1=pd.melt(temp_add, id_vars =['Markets', 'Products', 'Time_Period'], value_vars =Facts,var_name="Facts")

# changing datatype of "value" into "numeric"
df1["value"]=df1["value"].apply(pd.to_numeric)

# applying pivot_table ( Time_Period will come on columns and Facts will come on rows )
df2=df1.pivot_table(index=['Markets', 'Products','Facts'],columns='Time_Period')
df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

# showing dataframe df2
df2.head(3)

In [ ]:
# saving dataframe df2 to ADD_Data_final
ADD_Data_final=df2.copy()

# Creating "ShareBasis" column
ADD_Data_final["ShareBasis"]="ADD"

# Creating "Category" column
ADD_Data_final["Category"]="ADD"

# Creating "Manufacturer" column by replacing "MFR " to blank from "Products column"
ADD_Data_final["Manufacturer"]=[x.replace("MFR ","") if "MFR" in x else "" for x in ADD_Data_final["Products"]]

# Adding "ADD" in column "Products"
ADD_Data_final["Products"]=ADD_Data_final["Products"]+" ADD"

# Creating "Key" column by conacatinating columns "Markets","Products","Facts"
ADD_Data_final["Key"]=ADD_Data_final["Markets"]+ADD_Data_final["Products"]+ADD_Data_final["Facts"]

In [ ]:
print("UAP_Data ->",UAP_Data_final.shape,"\nADD_Data ->",ADD_Data_final.shape,"\nLine_Monthly ->",Line_Monthly.shape)

## Line Monthly with UAP&ADD Data

In [ ]:
Line_Monthly.columns

In [ ]:
del Line_Monthly_All

In [ ]:
# concat Line_Monthly,UAP_Data_final,ADD_Data_final dataframes in rows
Line_Monthly_All=pd.concat([Line_Monthly,UAP_Data_final,ADD_Data_final],axis=0,ignore_index=True, sort=False)

# Replacing Nan values with blank for New_Columns
New_Columns=["ShareBasis","Manufacturer","Franchise","Brand","SubBrand"]
Line_Monthly_All[New_Columns]=Line_Monthly_All[New_Columns].fillna(value="")

# Replacing Nan values with 0 for all dataframe
Line_Monthly_All.fillna(value=0,inplace=True)

print("Line_Monthly_All ->",Line_Monthly_All.shape)

In [ ]:
Line_Monthly_All.head(3)

In [ ]:
set(Line_Monthly_All["Category"])

## 1.Share Day Topline Line Monthly

Making a copy of ***Line_Monthly_All*** and storing it in ***SDT***

In [ ]:
SDT = Line_Monthly_All.copy()

We need **Brand** Level data for **ShareDay Topline Toolkit.**

**Removing all rows where SubBrand level data is available** after that **removing SubBrand column**

In [ ]:
#removing subbrand data
SDT.reset_index(drop=True,inplace=True)
index_names=SDT[SDT["SubBrand"]!=""].index
SDT.drop(index_names,inplace=True)

#removing subbrand column
col=list(SDT.columns)
col.remove("SubBrand")
SDT = SDT[col]

In [ ]:
SDT.head(2)

**Market, Facts, Time Period, ShareBasis and Products Selection for Share Day Topline**

Filter all the data based on the below selections for Share Day Topline

In [ ]:
Market =["REGIONAL GROCERY xLA","BJs Total TA","Dol Gen Total TA","Family Dollar Total TA","Sam's Total TA","Strategic Grocery","Target Total BM/FF TA","Total US Drug","Total US Food","Total US xAOC","Walmart Total US TA"]

Fact = ["$","$ % Chg YA","$ Shr - CP CATEGORY","$ Shr Chg YA - CP CATEGORY","$ Chg YA"]

Time_Period=list(SDT.columns[0:9])+list(SDT.columns[9:11])+list(SDT.columns[13:15])+list(SDT.columns[17:21])+list(SDT.columns[25:28])
#list(SDT.columns[9:11])

Share_Basis=["BAR","BODYWASH","BEAD FS","FABRIC SOFTENERS","LDL/DISHAIDS/WPS/FM/SPRAYS","LIQUID","LIQUID FS","MANUAL","MOUTHWASH",
             "POUR APC+WW","POWER","TOOTHPASTE","UAP","ADD"]

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
SDT=SDT[SDT['Markets'].isin(Market)]
SDT=SDT[SDT['Facts'].isin(Fact)]
SDT=SDT[SDT['ShareBasis'].isin(Share_Basis)]
SDT=SDT[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
SDT=SDT.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True])

# removing duplicates rows
SDT.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
SDT.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
SDT.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/ShareDay Topline_new.csv",index=False)

## 2.Executive Toolkit

### 2(a) ExecTool Data

Making a copy of ***Line_Monthly_All*** and storing it in ***ExecTool***

In [ ]:
ExecTool = Line_Monthly_All.copy()

We need **Brand** Level data for **Executive Toolkit.**

**Removing all rows where SubBrand level data is available** after that **removing SubBrand column**

In [ ]:
# removing subbrand data
ExecTool.reset_index(drop=True,inplace=True)
index_names=ExecTool[ExecTool["SubBrand"]!=""].index
ExecTool.drop(index_names,inplace=True)

# removing subbrand column
col=list(ExecTool.columns)
col.remove("SubBrand")
ExecTool = ExecTool[col]

# showing dataframe ExecTool
ExecTool.head(2)

**Market, Facts, Time Period, ShareBasis and Products Selection for Executive Toolkit**

Filter all the data based on the below selections for Executive Toolkit

In [ ]:
Market =["REGIONAL GROCERY xLA","BJs Total TA","CVS Total Corp WO HI TA","CLUB","Dol Gen Total TA","Family Dollar Total TA",
         "Publix Total TA","Rite Aid - Corp Total TA","Sam's Total TA","Strategic Grocery","Total US Drug","Target Total BM/FF TA",
         "Total US Food","Total US xAOC","VDC","Walgreens Total TA","Walmart Total US TA"]

Fact = ["$ Shr - CP CATEGORY","$ Shr Chg YA - CP CATEGORY","% $ Any Promo","% $ Any Promo Chg YA",'% EQ Any Promo',
        '% EQ Any Promo Chg YA','$','$ % Chg YA', '$ Chg YA', 'EQ','EQ % Chg YA',"EQ Chg YA","Any Promo $ Shr - CP CATEGORY",
        "Any Promo $ Shr Chg YA - CP CATEGORY","Units","Units % Chg YA","Units Chg YA","Avg EQ Price","Avg EQ Price % Chg YA",
        "Avg EQ Price Chg YA","% $ Any Promo","% $ Any Promo Chg YA","Base $","Base $ Chg YA","Incr $","Incr $ Chg YA"]

Time_Period=list(ExecTool.columns[0:9])+list(ExecTool.columns[9:11])+list(ExecTool.columns[13:14])+list(ExecTool.columns[15:17])+list(ExecTool.columns[25:28])

Share_Basis=["BAR","BODYWASH","LDL/DISHAIDS/WPS/FM/SPRAYS","LIQUID","LIQUID FS","MANUAL","MOUTHWASH",
             "POWER","POUR APC+WW","TOOTHPASTE","UAP","ADD"]

print("Time Period is ->\n",Time_Period)

In [ ]:
#filter data based on market, facts, time_period and share basis
ExecTool=ExecTool[ExecTool['Markets'].isin(Market)]
ExecTool=ExecTool[ExecTool['Facts'].isin(Fact)]
ExecTool=ExecTool[ExecTool['ShareBasis'].isin(Share_Basis)]
ExecTool=ExecTool[Time_Period]

In [ ]:
ExecTool.head(3)

In [ ]:
# sorting dataframe in markets, products, facts order
ExecTool=ExecTool.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True])   

# removing duplicates rows
ExecTool.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
ExecTool.fillna(0,inplace=True)

### 2(b) ExecToolkit xAOC and AHW Data

In [ ]:
# filtering ExecTool dataframe for "Total US xAOC" market
ExecTool_xAOC=ExecTool[ExecTool["Markets"]=="Total US xAOC"]

**Reading AHW Data**

In [ ]:
AHW_Data=pd.read_excel("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/AHW Data/AHW_Data.xlsx")

**Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

In [ ]:
new_col_names=[]
for i in AHW_Data.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))

# assigning new_col_names to AHW_Data
AHW_Data.columns=new_col_names

# assigning variables
LM_column= list(AHW_Data.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column=[]
for i in range(0,len(LM_column)):
    for j in range(0,len(time_connect)):
        if LM_column[i]==time_connect[j]:
            new_column.append(time_final[j])

#assigning new column names
AHW_Data.columns = new_column

In [ ]:
# Changing Column Name "Periods" to "Facts"
AHW_Data.rename(columns={"Periods":"Facts"},inplace=True)

# Creating "Key" column by conacatinating columns "Markets","Products","Facts"
AHW_Data["Key"]=AHW_Data["Markets"]+AHW_Data["Products"]+AHW_Data["Periods"]

# Sorting dataframe in markets, products, facts order
AHW_Datal=AHW_Data.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True])   

# Removing duplicates rows
AHW_Data.drop_duplicates(subset="Key",keep='last',inplace=True)

Appending **ExecTool_xAOC** and **AHW_Data**

In [ ]:
ExecTool_xAOC_AHW=pd.concat([ExecTool_xAOC,AHW_Data],axis=0)

Exporting Data to the output folder

In [ ]:
ExecTool_xAOC_AHW.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Executive Toolkit_xAOC.csv",index=False)

### 2(c) MFR Sheet Executive Toolkit

Making a copy of ***Line_Monthly_All*** and storing it in ***ExecTool***

In [ ]:
MFR_Sheet= Line_Monthly_All.copy()

We need **Manufacturer** Level data for **MFR Sheet Executive Toolkit.**

**Removing all rows where SubBrand,Brand,Franchise level data is available** after that **removing SubBrand,Brand,Franchise column**

In [ ]:
# Removing SubBrand,Brand,Franchise data
MFR_Sheet=MFR_Sheet.loc[(MFR_Sheet["SubBrand"]=="") & (MFR_Sheet["Brand"]=="")&(MFR_Sheet["Franchise"]=="")]

# Removing SubBrand,Brand,Franchise column
col=list(MFR_Sheet.columns)
col.remove("SubBrand")
col.remove("Brand")
col.remove("Franchise")
MFR_Sheet = MFR_Sheet[col]

# Showing Datafram
MFR_Sheet.head(2)

**Market, Facts, Time Period, ShareBasis and Products Selection for MFR Sheet Executive Toolkit**

Filter all the data based on the below selections for MFR Sheet Executive Toolkit

In [ ]:
Market =["Total US xAOC"]

Fact = ["$ % Chg YA","EQ % Chg YA","Avg EQ Price % Chg YA","$ Shr - CP CATEGORY"]

Time_Period=list(MFR_Sheet.columns[0:7])+list(MFR_Sheet.columns[7:12])+list(MFR_Sheet.columns[23:36])

Share_Basis=["BAR","BODYWASH","BEAD FS","BODYWASH","FABRIC SOFTENERS","LDL/DISHAIDS/WPS/FM/SPRAYS","LIQUID","LIQUID FS","MANUAL","MOUTHWASH",
             "POWER","POUR APC+WW","TOOTHPASTE","UAP","ADD"]

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
MFR_Sheet=MFR_Sheet[MFR_Sheet['Markets'].isin(Market)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['Facts'].isin(Fact)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['ShareBasis'].isin(Share_Basis)]
MFR_Sheet=MFR_Sheet[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
MFR_Sheet=MFR_Sheet.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
MFR_Sheet.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
MFR_Sheet.fillna(0,inplace=True)

In [ ]:
MFR_Sheet.head(2)

Exporting Data to the output folder

In [ ]:
MFR_Sheet.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Executive Toolkit_Manu.csv",index=False)

### 2(d) Rolling Executive Toolkit Part1

Making a copy of ***Line_Monthly_All*** and storing it in ***Roll***

In [ ]:
Roll=Line_Monthly_All.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Toolkit**

Filter all the data based on the below selections for Rolling Executive Toolkit

In [ ]:
Market=["Total US xAOC","Walmart Total US TA","Target Total BM/FF TA","Total US Food","Total US Drug",
        "CVS Total Corp WO HI TA","Walgreens Total TA","Sam's Total TA","Family Dollar Total TA","Dol Gen Total TA",
       "Rite Aid - Corp Total TA","Publix Total TA","BJs Total TA","Strategic Grocery","REGIONAL GROCERY xLA","CLUB","VDC"]

Facts =["$","$ Chg YA","Base $","Base $ Chg YA","Incr $","Incr $ Chg YA"]

ShareBasis=['ABRASIVE','BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'FABRIC SOFTENERS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW','PORTABLE TB', 'POWER', 'TOOTHPASTE',"UAP","ADD"]

OC=['TOOTHPASTE','MANUAL','POWER','PORTABLE TB','MOUTHWASH']
PC=['BAR', 'BODYWASH','LIQUID','UAP']
HC=[ 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID FS', 'POUR APC+WW','ADD','ABRASIVE']

Time_Period=list(Roll.columns[10:11])+list(Roll.columns[14:15])+list(Roll.columns[16:18])+list(Roll.columns[26:29])
print(Time_Period)


In [ ]:
# filter data based on market, facts, time_period and share basis
Roll_Cat=Roll.loc[(Roll["Manufacturer"]=="") & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(ShareBasis))]

In [ ]:
# Deleting unwanted columns
del Roll_Cat["Key"]
del Roll_Cat["Category"]
del Roll_Cat["Franchise"]
del Roll_Cat["Manufacturer"]
del Roll_Cat["Brand"]
del Roll_Cat["SubBrand"]
del Roll_Cat["Products"]

In [ ]:
# Creating column "Level" based on column "ShareBasis"
Roll_Cat["Level"]= ["OC" if x in OC else ("PC" if x in PC else "HC") for x in Roll_Cat["ShareBasis"]]

# Moving column "Level" after column "Markets"
Roll_Cat = movecol(Roll_Cat, 
             cols_to_move=['Level'], 
             ref_col='Markets',
             place='After')

**Rolling up all Facts** at **"Markets","Level"** level and creating **calculated fact $ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Level","Facts" for each Time_Period as sum
    df=Roll_Cat.groupby(["Markets","Level","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets","Level"], value_vars =list(df.columns[2:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Cat_final_1=df
    else:
        Roll_Cat_final_1=pd.merge(Roll_Cat_final_1,df,how='left',left_on=["Markets","Level","Facts"],right_on=["Markets","Level","Facts"])
                

**Rolling up all Facts** at **Market**  level and creating **calculated fact $ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Cat_final_1.groupby(["Markets","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets"], value_vars =list(df.columns[1:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Cat_final_2=df
    else:
        Roll_Cat_final_2=pd.merge(Roll_Cat_final_2,df,how='left',left_on=["Markets","Facts"],right_on=["Markets","Facts"])
                

In [ ]:
# Appending dataframes  Roll_Cat_final_1 and Roll_Cat_final_2
Roll_Cat_final=pd.concat([Roll_Cat_final_1,Roll_Cat_final_2],axis=0,ignore_index=True)

# Filling NaN values in "Level" column with "Total"( Toal includes all OC, PC, HC)
Roll_Cat_final["Level"].fillna("Total",inplace=True)

# Sorting the dataframe in order of "Markets","Level" and "Facts"
Roll_Cat_final.sort_values(['Markets', 'Level','Facts'],ascending=[True,True,True]) 

# Creating column "Key" as concat of "Markets","Level" and "Facts"
Roll_Cat_final["Key"]=Roll_Cat_final["Markets"]+Roll_Cat_final["Level"]+Roll_Cat_final["Facts"]

# Moving column "Key" before column "Markets"
Roll_Cat_final = movecol(Roll_Cat_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

# Saving final dataframe in "Executive_Roll_1"
Executive_Roll_1=Roll_Cat_final.copy()

Exporting Data to the output folder

In [ ]:
Roll_Cat_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Toolkit RollUp Part1.csv",index=False)

### 2(e) Rolling Executive Toolkit Part2

Making a copy of ***Line_Monthly_All*** and storing it in ***Roll***

In [ ]:
Roll=Line_Monthly_All.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Toolkit**

Filter all the data based on the below selections for Rolling Executive Toolkit

In [ ]:
Market=["Total US xAOC","Walmart Total US TA","Target Total BM/FF TA","Total US Food","Total US Drug",
        "CVS Total Corp WO HI TA","Walgreens Total TA","Sam's Total TA","Family Dollar Total TA","Dol Gen Total TA",
       "Rite Aid - Corp Total TA","Publix Total TA","BJs Total TA","Strategic Grocery","REGIONAL GROCERY xLA"]

Facts =["$","$ Chg YA","Base $","Base $ Chg YA","Incr $","Incr $ Chg YA"]

ShareBasis=['ABRASIVE','BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'FABRIC SOFTENERS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW','PORTABLE TB', 'POWER', 'TOOTHPASTE',"UAP","ADD"]

Time_Period=list(Roll.columns[10:11])+list(Roll.columns[14:15])+list(Roll.columns[16:18])+list(Roll.columns[26:29])
print(Time_Period)

OC=['TOOTHPASTE','MANUAL','POWER','PORTABLE TB','MOUTHWASH']
PC=['BAR', 'BODYWASH','LIQUID','UAP']
HC=[ 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID FS', 'POUR APC+WW','ADD','ABRASIVE']

Manu=["COLGATE-PALMOLIVE","P&G"]

In [ ]:
# filter data based on market, facts, time_period and share basis
Roll_Cat=Roll.loc[(Roll["Manufacturer"].isin(Manu)) & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(ShareBasis))]

In [ ]:
# Deleting unwanted columns
del Roll_Cat["Key"]
del Roll_Cat["Category"]
del Roll_Cat["Franchise"]
del Roll_Cat["Brand"]
del Roll_Cat["SubBrand"]
del Roll_Cat["Products"]

In [ ]:
# Creating column "Level" based on column "ShareBasis"
Roll_Cat["Level"]= ["OC" if x in OC else ("PC" if x in PC else "HC") for x in Roll_Cat["ShareBasis"]]

# Moving column "Level" after column "Markets"
Roll_Cat = movecol(Roll_Cat, 
             cols_to_move=['Level'], 
             ref_col='Markets',
             place='After')

**Rolling up all Facts** at **"Markets","Level","Manufacturer"** level and creating **calculated fact $ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Level","Manufacturer","Facts" for each Time_Period as sum
    df=Roll_Cat.groupby(["Markets","Level","Manufacturer","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
     
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets","Level","Manufacturer"], value_vars =list(df.columns[3:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Cat_final_1=df
    else:
        Roll_Cat_final_1=pd.merge(Roll_Cat_final_1,df,how='left',left_on=["Markets","Level","Manufacturer","Facts"],right_on=["Markets","Level","Manufacturer","Facts"])
                

**Rolling up all Facts** at **"Markets","Manufacturer"** level and creating **calculated fact $ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Manufacturer","Facts" for each Time_Period as sum
    df=Roll_Cat_final_1.groupby(["Markets","Manufacturer","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100

    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets","Manufacturer"], value_vars =list(df.columns[2:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns
    if (i == 0):
        Roll_Cat_final_2=df
    else:
        Roll_Cat_final_2=pd.merge(Roll_Cat_final_2,df,how='left',left_on=["Markets","Manufacturer","Facts"],right_on=["Markets","Manufacturer","Facts"])
                

In [ ]:
# Appending dataframes  Roll_Cat_final_1 and Roll_Cat_final_2
Roll_Cat_final=pd.concat([Roll_Cat_final_1,Roll_Cat_final_2],axis=0,ignore_index=True)

# Filling NaN values in "Level" column with "Total"( Toal includes all OC, PC, HC)
Roll_Cat_final["Level"].fillna("Total",inplace=True)

# Sorting the dataframe in order of "Markets","Level" and "Facts"
Roll_Cat_final.sort_values(['Markets', 'Level','Facts'],ascending=[True,True,True]) 

# Creating column "Key" as concat of "Markets","Level" and "Facts"
Roll_Cat_final["Key"]=Roll_Cat_final["Markets"]+Roll_Cat_final["Level"]+Roll_Cat_final["Manufacturer"]+Roll_Cat_final["Facts"]

# Moving column "Key" before column "Markets"
Roll_Cat_final = movecol(Roll_Cat_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')
# Saving final dataframe in "Executive_Roll_2"
Executive_Roll_2=Roll_Cat_final.copy()

Exporting Data to the output folder

In [ ]:
Roll_Cat_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Toolkit RollUp Part2.csv",index=False)

### 2(f) Adding Roll UP Data in Executive ToolkitData

In [ ]:
# Changing column "Level" name with "Products" for dataframes Executive_Roll_1 and Executive_Roll_2
Executive_Roll_1.rename(columns={"Level":"Products"},inplace=True)
Executive_Roll_2.rename(columns={"Level":"Products"},inplace=True)

# Appending Dataframe ExecTool,Executive_Roll_1,Executive_Roll_2
ExecTool_All = pd.concat([ExecTool,Executive_Roll_1,Executive_Roll_2],axis=0,ignore_index=True)

Exporting Data to the output folder

In [ ]:
ExecTool_All.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive_Toolkit_All.csv",index=False)

## 3. Executive Summary

### 3(a) Executive Summary

Making a copy of ***Line_Monthly_All*** and storing it in ***ExecSumm***

In [ ]:
ExecSumm = Line_Monthly_All.copy()

We need **Franchise** Level data for **Executive Summary.**

**Removing all rows where SubBrand,Brand level data is available** after that **removing SubBrand,Brand column**

In [ ]:
# removing brand data
ExecSumm.reset_index(drop=True,inplace=True)
index_names=ExecSumm[ExecSumm["Brand"]!=""].index
ExecSumm.drop(index_names,inplace=True)

# removing subbrand data
ExecSumm.reset_index(drop=True,inplace=True)
index_names=ExecSumm[ExecSumm["SubBrand"]!=""].index
ExecSumm.drop(index_names,inplace=True)

# removing brand,subbrand column
col=list(ExecSumm.columns)
col.remove("Brand")
col.remove("SubBrand")
ExecSumm = ExecSumm[col]

# showing dataframe ExecTool
ExecSumm.head(2)

**Market, Facts, Time Period, ShareBasis and Products Selection for Executive Summary**

Filter all the data based on the below selections for Executive Summary

In [ ]:
Market =["CLUB","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC","VDC","Walmart Total US TA",'Walgreens Total TA',"Rite Aid - Corp Total TA","Sam's Total TA","BJs Total TA","Family Dollar Total TA"]

Fact = ["$","$ % Chg YA","$ Chg YA",'EQ','EQ % Chg YA',"$ Shr - CP CATEGORY","$ Shr Chg YA - CP CATEGORY","EQ Shr - CP CATEGORY"
       ,"EQ Shr Chg YA - CP CATEGORY","Avg EQ Price","Avg EQ Price Chg YA"]

Time_Period=list(ExecSumm.columns[0:8])+list(ExecSumm.columns[8:11])+list(ExecSumm.columns[12:14])+list(ExecSumm.columns[16:36])

Share_Basis=["ABRASIVE","BAR","BODYWASH","FABRIC SOFTENERS","LDL/DISHAIDS/WPS/FM/SPRAYS","LIQUID","LIQUID FS","MANUAL","MOUTHWASH",
             "POWER","PORTABLE TB","POUR APC+WW","SHEETS","SCENT BOOSTERS","TOOTHPASTE","UAP","ADD"]

print("Time Period is ->\n",Time_Period)

In [ ]:
#filter data based on market, facts, time_period and share basis
ExecSumm=ExecSumm[ExecSumm['Markets'].isin(Market)]
ExecSumm=ExecSumm[ExecSumm['Facts'].isin(Fact)]
ExecSumm=ExecSumm[ExecSumm['ShareBasis'].isin(Share_Basis)]
ExecSumm=ExecSumm[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
ExecSumm=ExecSumm.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
ExecSumm.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
ExecSumm.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
ExecSumm.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Executive Summary.csv",index=False)

### 3(b) Rolling Executive Summary Part1

Making a copy of ***Line_Monthly_All*** and storing it in ***Roll***

In [ ]:
Roll= Line_Monthly_All.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Summary**

Filter all the data based on the below selections for Rolling Executive Summary

In [ ]:
Market=["Total US xAOC","Total US Drug","Total US Food","Walmart Total US TA","Target Total BM/FF TA",
        "TOTAL MASS","VDC","CLUB",'FDM']

ShareBasis=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'FABRIC SOFTENERS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE',"UAP"]

Core_Category=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'LIQ FS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE',"UAP"]

Adjacent_Category =["ABRASIVE","SCENT BOOSTERS","SHEETS","PORTABLE","ADD"]

Facts =["$","$ Chg YA","EQ","EQ Chg YA"]

Time_Period=list(Roll.columns[10:12])+list(Roll.columns[14:15])+list(Roll.columns[18:39])

print("Time Period ->\n", len(Time_Period),'\n',Time_Period)

#### 3(b).1  Roll up for ShareBasis

In [ ]:
# filter data based on market, facts, time_period and share basis
Roll_Market=Roll.loc[(Roll["Manufacturer"]=="") & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(ShareBasis))]

In [ ]:
# Deleting unwanted columns
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Manufacturer"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Rolling up all Facts** at **"Markets"** level and creating **calculated facts** ***$ % Chg YA*** and ***EQ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market.groupby(["Markets","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)

    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars ="Markets", value_vars =list(df.columns[1:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Market_final_1=df
    else:
        Roll_Market_final_1=pd.merge(Roll_Market_final_1,df,how='left',left_on=["Markets","Facts"],right_on=["Markets","Facts"])        

# Creting column "Level"
Roll_Market_final_1["Level"]="Total Category"

#### 3(b).2 Roll up for Core_Category

In [ ]:
# filter data based on market, facts, time_period and core_category
Roll_Market=Roll.loc[(Roll["Manufacturer"]=="") & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(Core_Category))]


In [ ]:
# Deleting unwanted columns
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Manufacturer"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Rolling up all Facts** at **"Markets"** level and creating **calculated facts** ***$ % Chg YA*** and ***EQ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market.groupby(["Markets","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars ="Markets", value_vars =list(df.columns[1:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Market_final_2=df
    else:
        Roll_Market_final_2=pd.merge(Roll_Market_final_2,df,how='left',left_on=["Markets","Facts"],right_on=["Markets","Facts"])
        
# Creting column "Level"
Roll_Market_final_2["Level"]="Core Category"

#### 3(b).3 Roll up for Adjacent_Category

In [ ]:
# filter data based on market, facts, time_period and adjacent_category
Roll_Market=Roll.loc[(Roll["Manufacturer"]=="") & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(Adjacent_Category))]


In [ ]:
# Deleting unwanted columns
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Manufacturer"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Rolling up all Facts** at **"Markets"** level and creating **calculated facts** ***$ % Chg YA*** and ***EQ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market.groupby(["Markets","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)

    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars ="Markets", value_vars =list(df.columns[1:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns
    if (i == 0):
        Roll_Market_final_3=df
    else:
        Roll_Market_final_3=pd.merge(Roll_Market_final_3,df,how='left',left_on=["Markets","Facts"],right_on=["Markets","Facts"])

# Creting column "Level"  
Roll_Market_final_3["Level"]="Adjacent Category"

In [ ]:
# concat all three Rollup files
Roll_Market_final=pd.concat([Roll_Market_final_1,Roll_Market_final_2,Roll_Market_final_3],axis=0,ignore_index=True)

# moving column "Level" after column "Markets"
Roll_Market_final = movecol(Roll_Market_final, 
             cols_to_move=['Level'], 
             ref_col='Markets',
             place='After')

# creating column "Key" by concatinating "Markets","Level","Facts"
Roll_Market_final["Key"]=Roll_Market_final["Markets"]+Roll_Market_final["Level"]+Roll_Market_final["Facts"]

# moving column "Key" before column "Markets"
Roll_Market_final = movecol(Roll_Market_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

# Creating column "Manufacturer"
Roll_Market_final["Manufacturer"]=""

# Moving column "Manufacturer" before column "Facts"
Roll_Market_final = movecol(Roll_Market_final, 
             cols_to_move=['Manufacturer'], 
             ref_col='Facts',
             place='Before')

# Sorting the dataframe in order of "Markets","Level" and "Facts"
Roll_Market_final.sort_values(['Markets', 'Level','Facts'],ascending=[True,True,True]) 

Exporting Data to the output folder

In [ ]:
Roll_Market_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Summary RollUp Part1.csv",index=False)

### 3(c) Rolling Executive Summary Part 2

Making a copy of ***Line_Monthly_All*** and storing it in ***Roll***

In [ ]:
Roll=Line_Monthly_All.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Summary**

Filter all the data based on the below selections for Rolling Executive Summary

In [ ]:
Market=["Total US xAOC","Total US Drug","Total US Food","Walmart Total US TA","Target Total BM/FF TA",
        "TOTAL MASS","VDC","CLUB",'FDM']
ShareBasis=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'FABRIC SOFTENERS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE']

Facts =["$","$ Chg YA","EQ","EQ Chg YA"]

Time_Period=list(Roll.columns[10:12])+list(Roll.columns[14:15])+list(Roll.columns[18:39])
print(Time_Period)

OC=['TOOTHPASTE','MANUAL','POWER','MOUTHWASH']
PC=['BAR', 'BODYWASH','LIQUID']
HC=[ 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID FS', 'POUR APC+WW']


In [ ]:
# filter data based on market, facts, time_period and share basis
Roll_Cat=Roll.loc[(Roll["Manufacturer"]=="") & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(ShareBasis))]

In [ ]:
# Deleting unwanted columns
del Roll_Cat["Key"]
del Roll_Cat["Category"]
del Roll_Cat["Franchise"]
del Roll_Cat["Manufacturer"]
del Roll_Cat["Brand"]
del Roll_Cat["SubBrand"]
del Roll_Cat["Products"]

In [ ]:
# Creating column "Level" based on column "ShareBasis"
Roll_Cat["Level"]= ["OC" if x in OC else ("PC" if x in PC else "HC") for x in Roll_Cat["ShareBasis"]]

# Moving column "Level" after column "Markets"
Roll_Cat = movecol(Roll_Cat, 
             cols_to_move=['Level'], 
             ref_col='Markets',
             place='After')

**Rolling up all Facts** at **"Markets","Level"** level and creating **calculated fact $ % Chg YA and EQ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Cat.groupby(["Markets","Level","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets","Level"], value_vars =list(df.columns[2:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Cat_final_1=df
    else:
        Roll_Cat_final_1=pd.merge(Roll_Cat_final_1,df,how='left',left_on=["Markets","Level","Facts"],right_on=["Markets","Level","Facts"])
                

**Rolling up all Facts** at **"Markets"** level and creating **calculated fact $ % Chg YA and EQ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Cat_final_1.groupby(["Markets","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets"], value_vars =list(df.columns[1:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns
    if (i == 0):
        Roll_Cat_final_2=df
    else:
        Roll_Cat_final_2=pd.merge(Roll_Cat_final_2,df,how='left',left_on=["Markets","Facts"],right_on=["Markets","Facts"])
                

In [ ]:
# Appending dataframes  Roll_Cat_final_1 and Roll_Cat_final_2
Roll_Cat_final=pd.concat([Roll_Cat_final_1,Roll_Cat_final_2],axis=0,ignore_index=True)

# Filling NaN values in "Level" column with "Total"
Roll_Cat_final["Level"].fillna("Total",inplace=True)

# Sorting the dataframe in order of "Markets","Level" and "Facts"
Roll_Cat_final.sort_values(['Markets', 'Level','Facts'],ascending=[True,True,True]) 

In [ ]:
# Creating column "Key" as concat of "Markets","Level" and "Facts"
Roll_Cat_final["Key"]=Roll_Cat_final["Markets"]+Roll_Cat_final["Level"]+Roll_Cat_final["Facts"]

# Moving column "Key" before column "Markets"
Roll_Cat_final = movecol(Roll_Cat_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

# Creating column "Manufacturer"
Roll_Cat_final["Manufacturer"]=""

# Moving column "Manufacturer" before column "Facts"
Roll_Cat_final = movecol(Roll_Cat_final, 
             cols_to_move=['Manufacturer'], 
             ref_col='Facts',
             place='Before')

Exporting Data to the output folder

In [ ]:
Roll_Cat_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Summary RollUp Part2.csv",index=False)

### 3(d) Rolling ExecSummary Part 3

Making a copy of ***Line_Monthly_All*** and storing it in ***Roll***

In [ ]:
Roll=Line_Monthly_All.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Summary**

Filter all the data based on the below selections for Rolling Executive Summary

In [ ]:
Market=["Total US xAOC","Total US Drug","Total US Food","Walmart Total US TA","Target Total BM/FF TA",
        "TOTAL MASS","VDC","CLUB",'FDM']

ShareBasis=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'FABRIC SOFTENERS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE']

Facts =["$","$ Chg YA","EQ","EQ Chg YA"]

Time_Period=list(Roll.columns[10:12])+list(Roll.columns[14:15])+list(Roll.columns[18:39])
print(Time_Period)

Manu=["COLGATE-PALMOLIVE","P&G"]

OC=['TOOTHPASTE','MANUAL','POWER','MOUTHWASH']
PC=['BAR', 'BODYWASH','LIQUID']
HC=[ 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID FS', 'POUR APC+WW']



In [ ]:
# filter data based on market, facts, time_period and share basis
Roll_Cat=Roll.loc[(Roll["Manufacturer"].isin(Manu)) & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(ShareBasis))]

In [ ]:
# Deleting unwanted columns
del Roll_Cat["Key"]
del Roll_Cat["Category"]
del Roll_Cat["Franchise"]
del Roll_Cat["Brand"]
del Roll_Cat["SubBrand"]
del Roll_Cat["Products"]

In [ ]:
# Creating column "Level" based on column "ShareBasis"
Roll_Cat["Level"]= ["OC" if x in OC else ("PC" if x in PC else "HC") for x in Roll_Cat["ShareBasis"]]

# Moving column "Level" after column "Markets"
Roll_Cat = movecol(Roll_Cat, 
             cols_to_move=['Level'], 
             ref_col='Markets',
             place='After')

**Rolling up all Facts** at **"Markets","Level","Manufacturer"** level and creating **calculated fact $ % Chg YA and EQ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Cat.groupby(["Markets","Level","Manufacturer","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets","Level","Manufacturer"], value_vars =list(df.columns[3:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns
    if (i == 0):
        Roll_Cat_final_1=df
    else:
        Roll_Cat_final_1=pd.merge(Roll_Cat_final_1,df,how='left',left_on=["Markets","Level","Manufacturer","Facts"],right_on=["Markets","Level","Manufacturer","Facts"])
                

**Rolling up all Facts** at **"Markets","Manufacturer"** level and creating **calculated fact $ % Chg YA and EQ % Chg YA**

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Cat_final_1.groupby(["Markets","Manufacturer","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets","Manufacturer"], value_vars =list(df.columns[2:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns
    if (i == 0):
        Roll_Cat_final_2=df
    else:
        Roll_Cat_final_2=pd.merge(Roll_Cat_final_2,df,how='left',left_on=["Markets","Manufacturer","Facts"],right_on=["Markets","Manufacturer","Facts"])
                

In [ ]:
# Appending dataframes  Roll_Cat_final_1 and Roll_Cat_final_2
Roll_Cat_final=pd.concat([Roll_Cat_final_1,Roll_Cat_final_2],axis=0,ignore_index=True)

# Filling NaN values in "Level" column with "Total"
Roll_Cat_final["Level"].fillna("Total",inplace=True)

# Sorting the dataframe in order of "Markets","Level" and "Facts"
Roll_Cat_final.sort_values(['Markets', 'Level','Facts'],ascending=[True,True,True]) 

# Creating column "Key" as concat of "Markets","Level" and "Facts"
Roll_Cat_final["Key"]=Roll_Cat_final["Markets"]+Roll_Cat_final["Level"]+Roll_Cat_final["Manufacturer"]+Roll_Cat_final["Facts"]

# Moving column "Key" before column "Markets"
Roll_Cat_final = movecol(Roll_Cat_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

Exporting Data to the output folder

In [ ]:
Roll_Cat_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Summary RollUp Part3.csv",index=False)

### 3(e) Rolling Executive Summary Part4

Making a copy of ***Line_Monthly_All*** and storing it in ***Roll***

In [ ]:
Roll=Line_Monthly_All.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Summary**

Filter all the data based on the below selections for Rolling Executive Summary

In [ ]:
Market=["Total US xAOC","Total US Drug","Total US Food","Walmart Total US TA","Target Total BM/FF TA",
        "TOTAL MASS","VDC","CLUB",'FDM']

ShareBasis=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'FABRIC SOFTENERS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE']

Core_Category=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'LIQ FS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE']

Adjacent_Category =["ABRASIVE","SCENT BOOSTERS","SHEETS","PORTABLE"]

Manu=["COLGATE-PALMOLIVE","P&G","PRVT LBL"]

Facts =["$","$ Chg YA","EQ","EQ Chg YA"]

Time_Period=list(Roll.columns[10:12])+list(Roll.columns[14:15])+list(Roll.columns[18:39])
print(Time_Period)


#### 3(e).1 Roll up for ShareBasis

In [ ]:
# filter data based on market, facts, time_period and share basis
Roll_Market=Roll.loc[(Roll["Manufacturer"].isin(Manu)) & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(ShareBasis))]

In [ ]:
# Deleting unwanted columns
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Rolling up all Facts** at **"Markets","Manufacturer** level and creating **calculated facts** ***$ % Chg YA*** and ***EQ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market.groupby(["Markets","Manufacturer","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets",'Manufacturer'], value_vars =list(df.columns[2:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Market_final_1=df
    else:
        Roll_Market_final_1=pd.merge(Roll_Market_final_1,df,how='left',left_on=["Markets","Manufacturer","Facts"],right_on=["Markets","Manufacturer","Facts"])
         
# Creting column "Level"
Roll_Market_final_1["Level"]="Total Category"

#### 3(e).2 Roll up for Core_Category

In [ ]:
# filter data based on market, facts, time_period and core_category
Roll_Market=Roll.loc[(Roll["Manufacturer"].isin(Manu)) & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(Core_Category))]


In [ ]:
# Deleting unwanted columns
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Rolling up all Facts** at **"Markets","Manufacturer"** level and creating **calculated facts** ***$ % Chg YA*** and ***EQ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market.groupby(["Markets",'Manufacturer',"Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
     # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
     # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets",'Manufacturer'], value_vars =list(df.columns[2:])) 
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Market_final_2=df
    else:
        Roll_Market_final_2=pd.merge(Roll_Market_final_2,df,how='left',left_on=["Markets",'Manufacturer',"Facts"],right_on=["Markets",'Manufacturer',"Facts"])
        
# Creting column "Level"
Roll_Market_final_2["Level"]="Core Category"

#### 3(e).3 Roll up for Adjacent_Category

In [ ]:
# filter data based on market, facts, time_period and adjacent_category
Roll_Market=Roll.loc[(Roll["Manufacturer"].isin(Manu)) & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
           (Roll["ShareBasis"].isin(Adjacent_Category))]


In [ ]:
# Deleting unwanted columns
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Rolling up all Facts** at **"Markets","Manufacturer** level and creating **calculated facts** ***$ % Chg YA*** and ***EQ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market.groupby(["Markets",'Manufacturer',"Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=(df["$ Chg YA"]/(df["$"]-df["$ Chg YA"]))*100
    # calculate "EQ % Chg YA"
    df["EQ % Chg YA"]=(df["EQ Chg YA"]/(df["EQ"]-df["EQ Chg YA"]))*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets",'Manufacturer'], value_vars =list(df.columns[2:]))  
    df.rename(columns={"value":Time_Period[i]},inplace=True)
    
    # merge dataframe for each time period on columns
    if (i == 0):
        Roll_Market_final_3=df
    else:
        Roll_Market_final_3=pd.merge(Roll_Market_final_3,df,how='left',left_on=["Markets",'Manufacturer',"Facts"],right_on=["Markets",'Manufacturer',"Facts"])

# Creting column "Level"
Roll_Market_final_3["Level"]="Adjacent Category"

In [ ]:
# concat all three Rollup files
Roll_Market_final=pd.concat([Roll_Market_final_1,Roll_Market_final_2,Roll_Market_final_3],axis=0,ignore_index=True)

# moving column "Level" after column "Markets"
Roll_Market_final = movecol(Roll_Market_final, 
             cols_to_move=['Level'], 
             ref_col='Markets',
             place='After')

# creating column "Key" by concatinating "Markets","Level","Facts"
Roll_Market_final.sort_values(['Markets', 'Level','Facts'],ascending=[True,True,True]) 

# creating column "Key" by concatinating "Markets","Level","Manufacturer","Facts"
Roll_Market_final["Key"]=Roll_Market_final["Markets"]+Roll_Market_final["Level"]+Roll_Market_final["Manufacturer"]+Roll_Market_final["Facts"]

# moving column "Key" before column "Markets"
Roll_Market_final = movecol(Roll_Market_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

Exporting Data to the output folder

In [ ]:
Roll_Market_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Summary RollUp Part4.csv",index=False)

### 3(f) Executive Summary Weekly Data

**Accessing Input Files**

In [ ]:
#Assigning path of the folder where **Line Weekly** data files are stored
path = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/Line Weekly"
files = os.listdir(path)

#Selecting only **excel files** from all the files available in the above path
files_xls = [f for f in files if ( f[-4:] == 'xlsx')]
print("Total number of files in the folder are ",len(files_xls))

**Appending the read files in rows and storing them in a single dataframe** named ***Line_Weekly***.

Printing number of columns and number of facts in each file.
Also checking the execution time of the code

In [ ]:
time_start= time.time()  # gives the current time

now=datetime.datetime.now()  # current date & time
delay= datetime.timedelta(days=30)   #30 days delay
month_name= (now - delay).strftime("%B")[0:3]+" " #30 days ago month name
print(month_name)


Line_Weekly= pd.DataFrame()  # creating a dataframe name Line_Weekly

j=0
for f in files_xls:
    
    data = pd.read_excel(path+"/"+f)    # Reading Data Files

    for i in data.columns:       # Checking if "Latest Month" column is available with another name. if yes then rename
        if month_name in i:
            data.rename(columns={i:"Latest Month"},inplace = True)

    # priniting number of facts and number of columns of the read file      
    print("\033[1m" + files_xls[j] + "\033[0m",":->\n# of Facts are ",len(set(data["Periods"])),"\t\t # of columns are ",len(data.columns))  

    # appending the read files in "Line_Weekly" dataframe
    Line_Weekly = pd.concat([Line_Weekly,data],axis=0,ignore_index=True)
    j=j+1
    
time_end= time.time(); # gives the current time

Backup= Line_Weekly.copy() # storing Line_Weekly dataframe into anoter dataframe for backup purpose 

# printing the time it takes to run this block of code
print("execution time is ", time_end-time_start)

**Removing Duplicate Rows**

In [ ]:
# Dropping the Duplicate rows
lenth_df=len(Line_Weekly)
Line_Weekly.drop_duplicates(subset=["Markets","Products","Periods"],keep='last',inplace=True)
print("Number of duplicated rows are",lenth_df-len(Line_Weekly),"\nShape of Line_Monthly is ",Line_Weekly.shape)

**Replacing Facts names** (Here column "Periods" denotes the fact column)

In [ ]:
# Replacing "- $ -" with "-" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("$","temp")
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("- temp -","-")
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("temp","$")

# Replacing "CP FORM" with "CP CATEGORY" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("CP FORM","CP CATEGORY")

# Replacing "CP USE" with "CP CATEGORY" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("CP USE","CP CATEGORY")

# Replacing "CP LDL AND FOAM" with "CP CATEGORY" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("CP LDL AND FOAM","CP CATEGORY")

# Replacing "CP LDL AND FM X DISHAIDS/WPS/SPR" with "CP CATEGORY" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("CP LDL AND FM X DISHAIDS/WPS/SPR","CP CATEGORY")

# Replacing "CP MEGA CATEGORY" with "CP CATEGORY" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("CP MEGA CATEGORY","CP CATEGORY")

# Replacing "CP BATTERY AND HYBRID" with "CP CATEGORY" in column "Periods"
Line_Weekly["Periods"]=Line_Weekly["Periods"].str.replace("CP BATTERY AND HYBRID","CP CATEGORY")


# Market name replaced
# Replacing "Walgreens Corp Total TA" with "Walgreens Total TA" in column "Markets"
Line_Weekly["Markets"]=Line_Weekly["Markets"].str.replace("Walgreens Corp Total TA","Walgreens Total TA")

**Renaming "Periods"** column name **to "Facts"**

In [ ]:
Line_Weekly.rename(columns={"Periods":"Facts"},inplace=True)

**Splitting "Products" column**

Creating 5 new columns **"ShareBasis","Manufacturer","Franchise","Brand","SubBrand"** from **"Products"** column by splitting 

In [ ]:
New_Columns=['ShareBasis','Manufacturer','Franchise','Brand','SubBrand']

Line_Weekly[New_Columns] = Line_Weekly['Products'].str.split('|',n=4,expand=True)

# Replacing all the None values in columns "ShareBasis","Manufacturer","Franchise","Brand","SubBrand"
Line_Weekly[New_Columns]=Line_Weekly[New_Columns].fillna(value="")
Line_Weekly.head(3)

# Moving columns after "Market"
Line_Weekly = movecol(Line_Weekly, 
             cols_to_move=['ShareBasis','Manufacturer','Franchise','Brand','SubBrand'], 
             ref_col='Markets',
             place='After')
Line_Weekly.head(3)

In [ ]:
# adding a "KEY" column
Line_Weekly["Key"]=Line_Weekly["Markets"]+Line_Weekly["Products"]+Line_Weekly["Facts"]

# moving "KEY" column to the first position by using above defined "movecol" function
Line_Weekly = movecol(Line_Weekly, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')
Line_Weekly.head(3)

**Strip Columns and Change Datatype to numeric**

Strip starting 9 columns 

Changing datatype to numeric after 9 columns

In [ ]:
#Storing the names of first 9 columns in a list
cols=list(Line_Weekly.columns[0:9])

#Removing blank space in all rows of the first 9 columns of the dataframe
Line_Weekly[cols]=Line_Weekly[cols].apply(lambda x : x.str.strip())

# Changing all columns after "Facts" into numeric datatype
Line_Weekly.iloc[:,10:]= Line_Weekly.iloc[:,10:].apply(pd.to_numeric)

**Category column**

Creating a extra column name "Category" from column "ShareBasis"

In [ ]:
#Creating a dataframe "df1" with only Sharebasis column
df1= pd.DataFrame(Line_Weekly["ShareBasis"])

In [ ]:
def fun_category(x):
    if x == 'TOOTHPASTE' or x == 'WHITENING SYS':
        return "TP"
    elif x=='POUR APC+WW' or x=="WOOD" or x== 'ABRASIVE' or x== "APC+WW" or x=='FLOOR WAX' or x=='OUTDOOR' or x=='DRY WIPES' or x=='SPRAY':
        return "HHC"
    elif x=='MANUAL':
        return 'MTB'
    elif x=='POWER' or x=='PORTABLE TB' or x == 'ELECTRIC' or x == 'BATTERY+HYBRID' or x == 'BATTERY':
        return "PTB"
    elif x=='BODYWASH':
        return "BW"
    elif x=='MOUTHWASH':
        return 'MW'
    elif x=='BAR':
        return "BAR"
    elif x=='LIQUID':
        return 'LHS'
    elif x=='FABRIC SOFTENERS' or x == 'BEAD FS' or x== 'LIQUID FS' or x == "SHEETS" or x=="SCENT BOOSTERS" or x == 'MFR Sun Products' :
        return "FS"
    elif x=='LDL+FOAM' or x == 'MFR Sun Products LDL' or x == "LDL/DISHAIDS/WPS/FM/SPRAYS":
        return 'LDL'
    else:
        return 'N/A'
    # Add elif statements....

df1['Category']= df1['ShareBasis'].apply(fun_category)

#Copying the Updated Category column to original Dataframe
Line_Weekly['Category']=df1['Category']

#Moving column "Category" after column "ShareBasis"
Line_Weekly = movecol(Line_Weekly, 
             cols_to_move=['Category'], 
             ref_col='ShareBasis',
             place='After')

In [ ]:
#Creating a copy of the working Dataset as a checkpoint
Line_Weekly_copy=Line_Weekly.copy()

### Roll Up Weekly

In [ ]:
# Creating a copy of the Dataframe
Roll=Line_Weekly.copy()

**Market, Facts, Time Period and ShareBasis Selection for Rolling Executive Summary**

Filter all the data based on the below selections for Rolling Executive Summary

In [ ]:
#Creating a list of required markets
Market=["Total US xAOC"]

#Creating a list of required categories
Core_Category=['BAR', 'BODYWASH', 'LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID', 'LIQUID FS', 'MANUAL', 'MOUTHWASH',
             'POUR APC+WW', 'POWER', 'TOOTHPASTE']

#Creating a list of required facts
Facts =["$","$ YA"]

Time_Period=list(Roll.columns[-13:])
print(Time_Period)

In [ ]:
#Filtering our Dataframe based on the above made lists and for "Colgate-Palmolive" manufacturer
Roll_Market=Roll.loc[(Roll["Manufacturer"]=="COLGATE-PALMOLIVE") & (Roll["Franchise"]=="") & (Roll["Brand"]=="") & 
           (Roll["SubBrand"]=="") & (Roll["Markets"].isin(Market))& (Roll["Facts"].isin(Facts)) &
            (Roll["ShareBasis"].isin(Core_Category))]

In [ ]:
#Deleting unwanted columns 
del Roll_Market["Key"]
del Roll_Market["Category"]
del Roll_Market["Manufacturer"]
del Roll_Market["Franchise"]
del Roll_Market["Brand"]
del Roll_Market["SubBrand"]
del Roll_Market["Products"]

**Reading UAP_Weekly file**

In [ ]:
UAP_Weekly=pd.read_excel("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/UAP_ADD_Data/UAP_Weekly.xlsx")

In [ ]:
#Renaming Period and Product columns
UAP_Weekly.rename(columns={"Periods":"Facts","Products":"ShareBasis"},inplace=True)

**Appending UAP data in Roll_Market**

In [ ]:
# Concating the UAP dataframe with the Roll_Market
Roll_Market_with_UAP = pd.concat([Roll_Market,UAP_Weekly],axis=0,ignore_index=True)

#Deleting duplicate rows 
Roll_Market_with_UAP.drop_duplicates(subset=["Markets","ShareBasis","Facts"],keep='last',inplace=True)

**Rolling up all Facts** at **"Markets"** level and creating **calculated facts** ***$ % Chg YA***

In [ ]:
for i in range(0,len(Time_Period)):
    
    # groupby "Markets","Facts" for each Time_Period as sum
    df=Roll_Market_with_UAP.groupby(["Markets","Facts"])[Time_Period[i]].aggregate('sum').unstack()
    df.reset_index(inplace=True)
    
    # calculate "$ % Chg YA"
    df["$ % Chg YA"]=((df["$"]-df["$ YA"])/df["$ YA"])*100
    
    # pivot each Fact from columns to a row
    df=pd.melt(df, id_vars =["Markets"], value_vars =list(df.columns[1:]),value_name=Time_Period[i]) 

    # merge dataframe for each time period on columns 
    if (i == 0):
        Roll_Market_final=df
    else:
        Roll_Market_final=pd.merge(Roll_Market_final,df,how='left',left_on=["Markets","Facts"],right_on=["Markets","Facts"])
               

In [ ]:
#sorting the dataframe based on Markets, Facts, respectively
Roll_Market_final.sort_values(['Markets','Facts'],ascending=[True,True]) 

#Adding a new column "Key" to the Dataframe
Roll_Market_final["Key"]=Roll_Market_final["Markets"]+Roll_Market_final["Facts"]

#Moving the "Key" column before the "market" column
Roll_Market_final = movecol(Roll_Market_final, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

Exporting Data to the output folder

In [ ]:
#Exporting the final datafraem as csv file to the desired location
Roll_Market_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data\Executive Summary RollUp Weekly.csv",index=False)

## 4. Category Toolkit OC

Making a copy of ***Line_Monthly_All*** and storing it in ***Cat_Tool***

In [ ]:
Cat_Tool=Line_Monthly_All.copy()

**Market, Facts, Time Period, ShareBasis and Products Selection for Category Toolkit OC**

Filter all the data based on the below selections for Category Toolkit OC

In [ ]:
Market =["BJs Total TA","CVS Total Corp WO HI TA","Dol Gen Total TA","Family Dollar Total TA","Publix Total TA",
         "Rite Aid - Corp Total TA","Sam's Total TA","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC",
         "Walmart Total US TA",'Walgreens Total TA']


Fact = ["$ Shr - CP CATEGORY","$ Shr Chg YA - CP CATEGORY",'EQ','EQ % Chg YA','Avg Unit Price',
        'Avg Unit Price Chg YA','Base $ Shr - CP CATEGORY','Base $ Shr Chg YA - CP CATEGORY'
       ,'Base EQ Price', 'Base EQ Price Chg YA',"Avg # of Items","Avg # of Items Chg YA","Incr $ Shr - CP CATEGORY","Incr $ Shr Chg YA - CP CATEGORY",
       'Incr $', 'Incr $ % Chg YA','Any Promo EQ Price', 'Any Promo EQ Price Chg YA','% $ Any Promo',
        '% $ Any Promo Chg YA','Any Promo $ Shr - CP CATEGORY', 'Any Promo $ Shr Chg YA - CP CATEGORY','Feat or Disp %ACV',
         'Feat or Disp %ACV Chg YA','$', '$ % Chg YA','Avg EQ Price', 'Avg EQ Price Chg YA',
        'Units', 'Units % Chg YA','%ACV','%ACV Chg YA','TDP','TDP Chg YA','Avg EQ Price % Chg YA','Base $ Chg YA',
       ]

Time_Period=list(Cat_Tool.columns[0:11])+list(Cat_Tool.columns[14:15])+list(Cat_Tool.columns[16:18])+list(Cat_Tool.columns[26:27])

Share_Basis=['ELECTRIC','MANUAL', 'MOUTHWASH','POWER' ,'TOOTHPASTE']

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
Cat_Tool=Cat_Tool[Cat_Tool['Markets'].isin(Market)]
Cat_Tool=Cat_Tool[Cat_Tool['Facts'].isin(Fact)]
Cat_Tool=Cat_Tool[Cat_Tool['ShareBasis'].isin(Share_Basis)]
Cat_Tool=Cat_Tool[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
Cat_Tool=Cat_Tool.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
Cat_Tool.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
Cat_Tool.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
Cat_Tool.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Category Toolkit OC_New.csv",index=False)

### 4(b) OC MFR SHEET

Making a copy of ***Line_Monthly_All*** and storing it in ***MFR_Sheet***

In [ ]:
MFR_Sheet= Line_Monthly.copy()

We need **Manufacturer** Level data for **MFR_Sheet.**

**Removing all rows where SubBrand,Brand,Franchise level data is available except "Dr Fresh" Franchise** after that **removing SubBrand,Brand,Level column**

In [ ]:
# removing subbrand data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["SubBrand"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing brand data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["Brand"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# taking "Dr Fresh" franchise for "High Ridge Brands" manufacturer
MFR_Sheet=MFR_Sheet[(MFR_Sheet["Franchise"].isin(["","DR FRESH"]))]
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[(MFR_Sheet["Manufacturer"]=="HIGH RIDGE BRANDS") & (MFR_Sheet["Franchise"]!="DR FRESH")].index
MFR_Sheet.drop(index_names,inplace=True)

# removing subbrand, brand, franchise columns
col=list(MFR_Sheet.columns)
col.remove("SubBrand")
col.remove("Brand")
col.remove("Franchise")
MFR_Sheet = MFR_Sheet[col]

**Market, Facts, Time Period, ShareBasis and Products Selection for MFR_Sheet**

Filter all the data based on the below selections for Category MFR_Sheet

In [ ]:
Market =["BJs Total TA","CVS Total Corp WO HI TA","Dol Gen Total TA","Family Dollar Total TA","Publix Total TA",
         "Rite Aid - Corp Total TA","Sam's Total TA","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC",
         "Walmart Total US TA",'Walgreens Total TA']

Fact= ["$ % Chg YA","EQ % Chg YA","Avg EQ Price Chg YA","Avg EQ Price % Chg YA"]

Time_Period=list(MFR_Sheet.columns[0:7])+list(MFR_Sheet.columns[23:36])

Share_Basis=['ELECTRIC','MANUAL', 'MOUTHWASH','POWER','TOOTHPASTE']

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
MFR_Sheet=MFR_Sheet[MFR_Sheet['Markets'].isin(Market)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['Facts'].isin(Fact)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['ShareBasis'].isin(Share_Basis)]
MFR_Sheet=MFR_Sheet[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
MFR_Sheet=MFR_Sheet.sort_alues(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
MFR_Sheet.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
MFR_Sheet.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
MFR_Sheet.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Category Toolkit OC Manu.csv",index=False)

## 5. Category Toolkit PC 

Making a copy of ***Line_Monthly_All*** and storing it in ***Cat_PC***

In [ ]:
Cat_PC=Line_Monthly_All.copy()

**Market, Facts, Time Period, ShareBasis and Products Selection for Cat_PC**

Filter all the data based on the below selections for Category Cat_PC

In [ ]:
Market =["BJs Total TA","CVS Total Corp WO HI TA","Dol Gen Total TA","Family Dollar Total TA","Publix Total TA",
         "Rite Aid - Corp Total TA","Sam's Total TA","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC",
         "Walmart Total US TA",'Walgreens Total TA']


Fact = ['$','$ % Chg YA','$ Shr - CP CATEGORY','$ Shr Chg YA - CP CATEGORY','% $ Any Promo','% $ Any Promo Chg YA',
        '%ACV','%ACV Chg YA','Any Promo $ Shr - CP CATEGORY','Any Promo $ Shr Chg YA - CP CATEGORY','Any Promo EQ Price',
        'Any Promo EQ Price Chg YA','Avg # of Items','Avg # of Items Chg YA','Avg EQ Price','Avg EQ Price % Chg YA',
        'Base $ Shr - CP CATEGORY','Base $ Shr Chg YA - CP CATEGORY','Base EQ Price','Base EQ Price Chg YA','EQ',
        'EQ % Chg YA','Feat or Disp %ACV','Feat or Disp %ACV Chg YA','Incr $ Shr - CP CATEGORY',
        'Incr $ Shr Chg YA - CP CATEGORY','TDP','TDP Chg YA']

Time_Period=list(Cat_PC.columns[0:11])+list(Cat_PC.columns[14:15])+list(Cat_PC.columns[16:18])+list(Cat_PC.columns[26:27])

Share_Basis=['BAR','BODYWASH', 'LIQUID','UAP']

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
Cat_PC=Cat_PC[Cat_PC['Markets'].isin(Market)]
Cat_PC=Cat_PC[Cat_PC['Facts'].isin(Fact)]
Cat_PC=Cat_PC[Cat_PC['ShareBasis'].isin(Share_Basis)]
Cat_PC=Cat_PC[Time_Period]

**Removing SubBrand Level Data**

In [ ]:
# removing subbrand data
Cat_PC=Cat_PC[Cat_PC["SubBrand"]==""]

# removing subbrand Column
col=list(Cat_PC.columns)
col.remove("SubBrand")
Cat_PC = Cat_PC[col]

In [ ]:
# sorting dataframe in markets, products, facts order
Cat_PC=Cat_PC.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
Cat_PC.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
Cat_PC.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
Cat_PC.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Category Toolkit PC.csv",index=False)

### 5(b) PC MFR Sheet

Making a copy of ***Line_Monthly_All*** and storing it in ***MFR_Sheet***

In [ ]:
MFR_Sheet= Line_Monthly_All.copy()

We need **Manufacturer** Level data for **MFR_Sheet.**

**Removing all rows where SubBrand,Brand,Franchise level data is available** after that **removing SubBrand,Brand,Level column**

In [ ]:
# removing subbrand data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["SubBrand"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing brand data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["Brand"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing franchise data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["Franchise"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing subbrand, brand, franchise columns
col=list(MFR_Sheet.columns)
col.remove("SubBrand")
col.remove("Brand")
col.remove("Franchise")
MFR_Sheet = MFR_Sheet[col]

**Market, Facts, Time Period, ShareBasis and Products Selection for MFR_Sheet**

Filter all the data based on the below selections for Category MFR_Sheet

In [ ]:
Market =["BJs Total TA","CVS Total Corp WO HI TA","Dol Gen Total TA","Family Dollar Total TA","Publix Total TA",
         "Rite Aid - Corp Total TA","Sam's Total TA","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC",
         "Walmart Total US TA",'Walgreens Total TA']


Fact= ["$ % Chg YA","EQ % Chg YA","Avg EQ Price Chg YA","Avg EQ Price % Chg YA"]

Time_Period=list(MFR_Sheet.columns[0:7])+list(MFR_Sheet.columns[23:36])

Share_Basis=["BAR","BODYWASH","LIQUID","UAP"]

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
MFR_Sheet=MFR_Sheet[MFR_Sheet['Markets'].isin(Market)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['Facts'].isin(Fact)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['ShareBasis'].isin(Share_Basis)]
MFR_Sheet=MFR_Sheet[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
MFR_Sheet=MFR_Sheet.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
MFR_Sheet.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
MFR_Sheet.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
MFR_Sheet.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Category Toolkit PC Manu.csv",index=False)

## 6. Category Toolkit HC

Making a copy of ***Line_Monthly_All*** and storing it in ***Cat_HC***

In [ ]:
Cat_HC=Line_Monthly_All.copy()

**Market, Facts, Time Period, ShareBasis and Products Selection for Cat_HC**

Filter all the data based on the below selections for Category Cat_HC

In [ ]:
Market =["BJs Total TA","CVS Total Corp WO HI TA","Dol Gen Total TA","Family Dollar Total TA","Publix Total TA",
         "Rite Aid - Corp Total TA","Sam's Total TA","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC",
         "Walmart Total US TA",'Walgreens Total TA']


Fact = ['$','$ % Chg YA','$ Shr - CP CATEGORY','$ Shr Chg YA - CP CATEGORY','% $ Any Promo','% $ Any Promo Chg YA',
        '%ACV','%ACV Chg YA','Any Promo $ Shr - CP CATEGORY','Any Promo $ Shr Chg YA - CP CATEGORY','Any Promo EQ Price',
        'Any Promo EQ Price Chg YA','Avg # of Items','Avg # of Items Chg YA','Avg EQ Price','Avg EQ Price % Chg YA',
        'Base $ Shr - CP CATEGORY','Base $ Shr Chg YA - CP CATEGORY','Base EQ Price','Base EQ Price Chg YA','EQ',
        'EQ % Chg YA','Feat or Disp %ACV','Feat or Disp %ACV Chg YA','Incr $ Shr - CP CATEGORY',
        'Incr $ Shr Chg YA - CP CATEGORY','TDP','TDP Chg YA']

Time_Period=list(Cat_HC.columns[0:10])+list(Cat_HC.columns[10:11])+list(Cat_HC.columns[14:15])+list(Cat_HC.columns[16:18])+list(Line_Monthly.columns[26:27])

Share_Basis=['BEAD FS','FABRIC SOFTENERS','LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID FS','POUR APC+WW','SHEETS',"SCENT BOOSTERS","ADD","MFR Sun Products","MFR Sun Products LDL"]

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
Cat_HC=Cat_HC[Cat_HC['Markets'].isin(Market)]
Cat_HC=Cat_HC[Cat_HC['Facts'].isin(Fact)]
Cat_HC=Cat_HC[Cat_HC['ShareBasis'].isin(Share_Basis)]
Cat_HC=Cat_HC[Time_Period]

**Removing SubBrand Level Data and taking Manufacturer level data for ShareBasis "Bead FS","Fabric Softners","Sheets","Scent Boosters"**

In [ ]:
# removing subbrand data
Cat_HC_final=Cat_HC[Cat_HC["SubBrand"]==""]

# Manufacturer level data for sharebasis "Bead FS","Fabric Softners","Sheets","Scent Boosters"
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=Cat_HC_final[(Cat_HC_final["ShareBasis"].isin(["BEAD FS","FABRIC SOFTENERS","SHEETS","SCENT BOOSTERS"]) &(Cat_HC_final["Franchise"]!=""))].index
Cat_HC_final.drop(index_names,inplace=True)

# removing subbrand column
col=list(Cat_HC_final.columns)
col.remove("SubBrand")
Cat_HC_final = Cat_HC_final[col]

In [ ]:
# sorting dataframe in markets, products, facts order
Cat_HC=Cat_HC.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
Cat_HC.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
Cat_HC.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
Cat_HC.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Category Toolkit HC.csv",index=False)

### 6(b) HC MFR Sheet

Making a copy of ***Line_Monthly_All*** and storing it in ***MFR_Sheet***

In [ ]:
MFR_Sheet= Line_Monthly_All.copy()

We need **Manufacturer** Level data for **MFR_Sheet.**

**Removing all rows where SubBrand,Brand,Franchise level data is available** after that **removing SubBrand,Brand,Level column**

In [ ]:
# removing subbrand data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["SubBrand"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing brand data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["Brand"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing franchise data
MFR_Sheet.reset_index(drop=True,inplace=True)
index_names=MFR_Sheet[MFR_Sheet["Franchise"]!=""].index
MFR_Sheet.drop(index_names,inplace=True)

# removing subbrand, brand, franchise columns
col=list(MFR_Sheet.columns)
col.remove("SubBrand")
col.remove("Brand")
col.remove("Franchise")
MFR_Sheet = MFR_Sheet[col]

**Market, Facts, Time Period, ShareBasis and Products Selection for MFR_Sheet**

Filter all the data based on the below selections for Category MFR_Sheet

In [ ]:
Market =["BJs Total TA","CVS Total Corp WO HI TA","Dol Gen Total TA","Family Dollar Total TA","Publix Total TA",
         "Rite Aid - Corp Total TA","Sam's Total TA","Total US Drug","Target Total BM/FF TA","Total US Food","Total US xAOC",
         "Walmart Total US TA",'Walgreens Total TA']

Fact= ["$ % Chg YA","EQ % Chg YA","Avg EQ Price Chg YA","Avg EQ Price % Chg YA"]

Time_Period=list(MFR_Sheet.columns[23:36])

Share_Basis=['BEAD FS','FABRIC SOFTENERS','LDL/DISHAIDS/WPS/FM/SPRAYS', 'LIQUID FS','POUR APC+WW','SHEETS',"ADD","MFR Sun Products","MFR Sun Products LDL"]

print("Time Period is ->\n",Time_Period)

In [ ]:
# filter data based on market, facts, time_period and share basis
MFR_Sheet=MFR_Sheet[MFR_Sheet['Markets'].isin(Market)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['Facts'].isin(Fact)]
MFR_Sheet=MFR_Sheet[MFR_Sheet['ShareBasis'].isin(Share_Basis)]
MFR_Sheet=MFR_Sheet[Time_Period]

In [ ]:
# sorting dataframe in markets, products, facts order
MFR_Sheet=MFR_Sheet.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True]) 

# removing duplicates rows
MFR_Sheet.drop_duplicates(subset="Key",keep='last',inplace=True)

# replacing nan values from zero
MFR_Sheet.fillna(0,inplace=True)

Exporting Data to the output folder

In [ ]:
MFR_Sheet.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/Category Toolkit HC Manu.csv",index=False)